In [11]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K

#------------用于绘制模型细节，可选--------------#
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#------------------------------------------------#

K.set_image_data_format('channels_first')

import time
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import fr_utils
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

#np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 简单的人脸验证
在人脸验证中，你需要给出两张照片并想知道是否是同一个人，最简单的方法是逐像素地比较这两幅图像，如果图片之间的误差小于选择的阈值，那么则可能是同一个人。 
<img src="1.png" style="width:380px;height:150px;">
当然，如果你真的这么做的话效果一定会很差，因为像素值的变化在很大程度上是由于光照、人脸的朝向、甚至头部的位置的微小变化等等。接下来与使用原始图像不同的是我们可以让系统学习构建一个编码$f(img)$，对该编码的元素进行比较，可以更准确地判断两幅图像是否属于同一个人。

# 将人脸图像编码为128位的向量
## 使用卷积网络来进行编码
FaceNet模型需要大量的数据和长时间的训练，因为，遵循在应用深度学习设置中常见的实践，我们要加载其他人已经训练过的权值。在网络的架构上我们遵循Szegedy et al.等人的初始模型。这里我们提供了初始模型的实现方法，你可以打开inception_blocks.py文件来查看是如何实现的。

关键信息如下：

- 该网络使用了96×9696×96的RGB图像作为输入数据，图像数量为mm，输入的数据维度为$(m, n_C, n_H, n_W) = (m, 3, 96, 96)$ 

- 输出为$(m,128)$的已经编码的mm个128128位的向量。

In [12]:
#获取模型
FRmodel = faceRecoModel(input_shape=(3,96,96))

#打印模型的总参数数量
print("参数数量：" + str(FRmodel.count_params()))

Instructions for updating:
Colocations handled automatically by placer.
参数数量：3743280


通过使用128神经元全连接层作为最后一层，该模型确保输出是大小为128的编码向量，然后使用比较两个人脸图像的编码如下：
<img src="2.png" style="width:500px;height:200px;">
因此，如果满足下面两个条件的话，编码是一个比较好的方法：

- 同一个人的两个图像的编码非常相似。

- 两个不同人物的图像的编码非常不同。

 三元组损失函数将上面的形式实现，它会试图将同一个人的两个图像（对于给定的图和正例）的编码“拉近”，同时将两个不同的人的图像（对于给定的图和负例）进一步“分离”。
 <img src="3.png" style="width:500px;height:200px;">

## 三元组损失函数
对于给定的图像$x$，其编码为$f(x)$，其中$f$为神经网络的计算函数
我们将使用三元组图像$（A，P，N）$进行训练：

- A是“Anchor”，是一个人的图像。

- P是“Positive”，是相对于“Anchor”的同一个人的另外一张图像。

- N是“Negative”，是相对于“Anchor”的不同的人的另外一张图像。
这些三元组来自训练集，我们使用$(A(i),P(i),N(i))$来表示第$i$个训练样本。我们要保证图像$A(i)$与图像$P(i)$的差值至少比与图像$N(i)$的差值相差$\alpha$：
$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$
我们希望让三元组损失变为最小：
$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$
- 在这里，我们使用“[⋅⋅⋅]+”来表示函数max(z,0)
- $\alpha$是间距，这个需要我们来手动选择，这里我们使用$\alpha$=0.2

In [13]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    根据公式（4）实现三元组损失函数

    参数：
        y_true -- true标签，当你在Keras里定义了一个损失函数的时候需要它，但是这里不需要。
        y_pred -- 列表类型，包含了如下参数：
            anchor -- 给定的“anchor”图像的编码，维度为(None,128)
            positive -- “positive”图像的编码，维度为(None,128)
            negative -- “negative”图像的编码，维度为(None,128)
        alpha -- 超参数，阈值

    返回：
        loss -- 实数，损失的值
    """
    #获取anchor, positive, negative的图像编码
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    #第一步：计算"anchor" 与 "positive"之间编码的距离，这里需要使用axis=-1
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)

    #第二步：计算"anchor" 与 "negative"之间编码的距离，这里需要使用axis=-1
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)

    #第三步：减去之前的两个距离，然后加上alpha
    basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)

    #通过取带零的最大值和对训练样本的求和来计算整个公式
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))

    return loss

In [14]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)

    print("loss = " + str(loss.eval()))

loss = 528.1432


# 加载训练好了的模型

In [15]:
#开始时间
start_time = time.clock()

#编译模型
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

#加载权值
fr_utils.load_weights_from_FaceNet(FRmodel)

#结束时间
end_time = time.clock()

#计算时差
minium = end_time - start_time

print("执行了：" + str(int(minium / 60)) + "分" + str(int(minium%60)) + "秒")

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


执行了：2分23秒


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # This is added back by InteractiveShellApp.init_path()


# 模型的应用
## 人脸验证
我们构建一个数据库，里面包含了允许进入的人员的编码向量，我们使用fr_uitls.img_to_encoding(image_path, model)函数来生成编码，它会根据图像来进行模型的前向传播。 
 我们这里的数据库使用的是一个字典来表示，这个字典将每个人的名字映射到他们面部的128维编码上。

In [16]:
database = {}
database["danielle"] = fr_utils.img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = fr_utils.img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = fr_utils.img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = fr_utils.img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = fr_utils.img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = fr_utils.img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = fr_utils.img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = fr_utils.img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = fr_utils.img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = fr_utils.img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = fr_utils.img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = fr_utils.img_to_encoding("images/arnaud.jpg", FRmodel)

现在，当有人出现在你的门前刷他们的身份证的时候，你可以在数据库中查找他们的编码，用它来检查站在门前的人是否与身份证上的名字匹配。

 现在我们要实现 verify() 函数来验证摄像头的照片(image_path)是否与身份证上的名称匹配，这个部分可由以下步骤构成：

- 根据image_path来计算编码。

- 计算与存储在数据库中的身份图像的编码的差距。

- 如果差距小于0.7，那么就打开门，否则就不开门。

 如上所述，我们使用L2(np.linalg.norm)来计算差距。(注意:在本实现中，将L2的误差(而不是L2误差的平方)与阈值0.7进行比较。)

In [18]:
def verify(image_path, identity, database, model):
    """
    对“identity”与“image_path”的编码进行验证。

    参数：
        image_path -- 摄像头的图片。
        identity -- 字符类型，想要验证的人的名字。
        database -- 字典类型，包含了成员的名字信息与对应的编码。
        model -- 在Keras的模型的实例。

    返回：
        dist -- 摄像头的图片与数据库中的图片的编码的差距。
        is_open_door -- boolean,是否该开门。
    """
    #第一步：计算图像的编码，使用fr_utils.img_to_encoding()来计算。
    encoding = fr_utils.img_to_encoding(image_path, model)

    #第二步：计算与数据库中保存的编码的差距
    dist = np.linalg.norm(encoding - database[identity])

    #第三步：判断是否打开门
    if dist < 0.7:
        print("欢迎 " + str(identity) + "回家！")
        is_door_open = True
    else:
        print("经验证，您与" + str(identity) + "不符！")
        is_door_open = False

    return dist, is_door_open

In [19]:
verify("images/camera_0.jpg","younes",database,FRmodel)

欢迎 younes回家！


(0.6710072, True)

In [20]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

经验证，您与kian不符！


(0.8580013, False)

## 人脸识别
面部验证系统基本运行良好，但是自从Kian的身份证被偷后，那天晚上他回到房子那里就不能进去了!为了减少这种恶作剧，你想把你的面部验证系统升级成面部识别系统。这样就不用再带身份证了，一个被授权的人只要走到房子前面，前门就会自动为他们打开!

 我们将实现一个人脸识别系统，该系统将图像作为输入，并确定它是否是授权人员之一(如果是，是谁),与之前的人脸验证系统不同，我们不再将一个人的名字作为输入的一部分。

 现在我们要实现who_is_it()函数，实现它需要有以下步骤：

- 根据image_path计算图像的编码。

- 从数据库中找出与目标编码具有最小差距的编码。

   - 初始化min_dist变量为足够大的数字（100），它将找到与输入的编码最接近的编码。
   - 遍历数据库中的名字与编码，可以使用for (name, db_enc) in database.items()语句。 
       - 计算目标编码与当前数据库编码之间的L2差距。
       - 如果差距小于min_dist，那么就更新名字与编码到identity与min_dist中。

In [22]:
def who_is_it(image_path, database,model):
    """
    根据指定的图片来进行人脸识别

    参数：
        images_path -- 图像地址
        database -- 包含了名字与编码的字典
        model -- 在Keras中的模型的实例。

    返回：
        min_dist -- 在数据库中与指定图像最相近的编码。
        identity -- 字符串类型，与min_dist编码相对应的名字。
    """
    #步骤1：计算指定图像的编码，使用fr_utils.img_to_encoding()来计算。
    encoding = fr_utils.img_to_encoding(image_path, model)

    #步骤2 ：找到最相近的编码
    ## 初始化min_dist变量为足够大的数字，这里设置为100
    min_dist = 100

    ## 遍历数据库找到最相近的编码
    for (name,db_enc) in database.items():
        ### 计算目标编码与当前数据库编码之间的L2差距。
        dist = np.linalg.norm(encoding - db_enc)

        ### 如果差距小于min_dist，那么就更新名字与编码到identity与min_dist中。
        if dist < min_dist:
            min_dist = dist
            identity = name

    # 判断是否在数据库中
    if min_dist > 0.7:
        print("抱歉，您的信息不在数据库中。")

    else:
        print("姓名" + str(identity) + "  差距：" + str(min_dist))

    return min_dist, identity

In [23]:
who_is_it("images/camera_0.jpg", database, FRmodel)

姓名younes  差距：0.6710072


(0.6710072, 'younes')